<a href="https://colab.research.google.com/github/deejay14/Jhakas-NewsWala/blob/main/LSH_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools

import matplotlib.pyplot as plt
%matplotlib inline

In [84]:
 pip install dateparser

In [85]:
url='https://raw.githubusercontent.com/deejay14/Jhakas-NewsWala/main/final_fixed_scrapped.csv'    #reads final document
df = pd.read_csv(url) 

In [86]:
df

,Unnamed: 0,Title,Article,totalwords
0,0,Rs 400-cr ‘bungling’ by Amar Singh: HC seeks E...,The Allahabad High Court has directed the Enfo...,341
1,1,J&K HC orders FIR against teacher who fails to...,The J&K High Court on Friday asked a teacher t...,335
2,2,Crime in Cyberia: an incomplete list of offenders,An unknown Indian hacker has been charged with...,382
3,3,450 cannabis plants found on govt land in Mundhwa,The interrogation of two marijuana peddlers ab...,372
4,4,"Nagaland: 9 bullet-riddled bodies found,Karbi-...",Nine highly decomposed bodies were found stack...,449
...,...,...,...,...
7838,7838,"13 dead, one million evacuated in China after ...",At least 18 people were killed due to a landsl...,397
7839,7839,"Book Karnataka, Maharashtra CMs for culpable h...",With the flood situation in the state continui...,500
7840,7840,Romanian researcher develops disc-shaped flyin...,"Razan Sabie, an aerodynamicist who operates fr...",346
7841,7841,Is asteroid Apophis capable of triggering a ts...,It was around a few months back that Iain McDo...,412


In [87]:
df['Article']=df['Article'].apply(str)
data_list = df['Article'].values.tolist()

df['Title']=df['Title'].apply(str)
data_titles = df['Title'].values.tolist()

data_list[:1]

['The Allahabad High Court has directed the Enforcement Directorate (ED) to file a status report, along with an affidavit, in connection with the investigation carried out in connection with the allegations of financial misappropriation to the tune over Rs 400 crore by former Samajwadi Party leader Amar Singh. The court has fixed October 8 for next hearing.A Division Bench of Chief Justice D Y Chandrachud and Justice Yashwant Varma passed the order on Thursday, while hearing a PIL filed by Ashok Kumar Singh, a resident of Kanpur, through his counsel Ashok Pandey.The petitioner, who was at one point district chief of SP in Kanpur, had pointed out that a three-judge bench had, in its order on August 1, 2013, directed further investigation by the ED in connection with the allegations against Amar Singh. It was alleged that as the head of the UP development council, he had misappropriated funds to the amount of Rs 400 crore. An FIR in this regard was registered in Kanpur on October 15, 200

Data Cleaning

In [88]:
import re                                                                       # usual lemmetisation
from pprint import pprint
import nltk, spacy, gensim
from sklearn.feature_extraction.text import CountVectorizer

def get_lemmatized_clean_data(df):
    # Convert to list
    data = df['Article'].values.tolist()

    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    pprint(data[:1])

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(data))

    print(data_words[:1])
    
    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
        return texts_out

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # Run in terminal: python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    # Do lemmatization keeping only Noun, Adj, Verb, Adverb
    data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    return data_lemmatized

In [89]:
data_lemmatized = get_lemmatized_clean_data(df)

['The Allahabad High Court has directed the Enforcement Directorate (ED) to '
 'file a status report, along with an affidavit, in connection with the '
 'investigation carried out in connection with the allegations of financial '
 'misappropriation to the tune over Rs 400 crore by former Samajwadi Party '
 'leader Amar Singh. The court has fixed October 8 for next hearing.A Division '
 'Bench of Chief Justice D Y Chandrachud and Justice Yashwant Varma passed the '
 'order on Thursday, while hearing a PIL filed by Ashok Kumar Singh, a '
 'resident of Kanpur, through his counsel Ashok Pandey.The petitioner, who was '
 'at one point district chief of SP in Kanpur, had pointed out that a '
 'three-judge bench had, in its order on August 1, 2013, directed further '
 'investigation by the ED in connection with the allegations against Amar '
 'Singh. It was alleged that as the head of the UP development council, he had '
 'misappropriated funds to the amount of Rs 400 crore. An FIR in this re

In [90]:
data_lemmatized[20]

'explain why most respected pm state party would never align bjp idea exchange moderate editor chhibber pdp president also express disappointment say should give chance prove claim come hate government rumour join hand bjp will talk recent statement chief minister go say pdp pakistani party look flag say father doesn come point always praise atal vajpayee change see today go lahore invite take in come say few touching word go resolve issue extend hand friendship work alliance partner contrast national conference kick saifuddin soz when vote then become minister vajpayee government present when happen only defend modi breaking point come when announce go time frame poll expect experience have alliance party centre different when form alliance able manipulate leadership rig srinagar election pdp stand more empowerment self rule other thing resolution problem aversion article think meeting point rakesh sinha view modi find modi very authoritative seem grow much more party perhaps people l

In [91]:
len(data_lemmatized)

7843

Tokenizer:

In [92]:
def tokenizer(sentences):
  def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
  tokens=list(sent_to_words(sentences))
  return tokens


In [93]:
check=tokenizer(data_lemmatized)
len(check)

7843

In [94]:
pip install datasketch

Minhash implementation : Creation of Forest (equivalent to training in LDA)

In [95]:
import time
from datasketch import MinHash, MinHashLSHForest

permutations = 128

#Number of Recommendations to return
num_recommendations = 1
def get_forest(perms):
    start_time = time.time()
    
    minhash = []
    
    for tok in check:
      m = MinHash(num_perm=perms)
      for s in tok:
          m.update(s.encode('utf8'))
      minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

Generating quey in the next two cells:


In [103]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

texta=preprocess(data_lemmatized[5])



In [104]:
print(texta)

['today', 'election', 'hold', 'opposition', 'boycott', 'shubhajit', 'find', 'country', 'seek', 'retain', 'identity', 'grow', 'islamic', 'influence', 'unfinished', 'business', 'mamoon', 'worried', 'man', 'today', 'year', 'old', 'history', 'professor', 'warn', 'child', 'would', 'snap', 'relation', 'follow', 'extremist', 'ideology', 'spread', 'political', 'force', 'country', 'bangladesh', 'head', 'poll', 'today', 'elect', 'th', 'parliament', 'mamoon', 'many', 'other', 'citizen', 'country', 'election', 'usual', 'issue', 'corruption', 'economy', 'law', 'order', 'election', 'centre', 'very', 'idea', 'bangladesh', 'should', 'secular', 'democracy', 'theocratic', 'state', 'islamic', 'nation', 'protest', 'call', 'opposition', 'bnp', 'align', 'islamist', 'more', 'people', 'kill', 'clash', 'past', 'half', 'month', 'bnp', 'boycott', 'poll', 'call', 'farce', 'ask', 'people', 'stay', 'away', 'statement', 'when', 'call', 'hour', 'begin', 'morning', 'include', 'polling', 'day', 'bnp', 'say', 'one', 'ho

**Prediction Process :**

In [98]:
def predict(text, df, perms, num_results, forest):
    start_time = time.time()
    
    tokens = text
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = df.iloc[idx_array]['Article']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [82]:
forest = get_forest(permutations)

It took 36.946781635284424 seconds to build forest.


In [105]:
num_recommendations = 10
result = predict(texta, df, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.023380517959594727 seconds to query forest.

 Top Recommendation(s) is(are) 
 4224    About this series — \nAh, 2020: The year when ...
5       With todays elections being held under an Opp...
6       With todays elections being held under an Opp...
1030    Retd Delhi HC Chief Justice and the man behind...
1095    In a curious submission,the government on Frid...
2322    The Goa Police has sought additional security ...
2007    Twenty-seven South Asian workers,including 24 ...
4154    Read more from our '2020, the year of...' seri...
604     Fair,transparent court martial proceedings wou...
4478    By Meredith Jones\nKim Kardashian West has ann...
Name: Article, dtype: object


In [113]:
  print(data_lemmatized[5])

today election hold opposition boycott shubhajit find country seek retain identity grow islamic influence unfinished business mamoon worried man today year old history professor warn child would snap relation follow extremist ideology spread political force country bangladesh head poll today elect th parliament mamoon many other citizen country election usual issue corruption economy law order election centre very idea bangladesh should secular democracy theocratic state islamic nation protest call opposition bnp align islamist more people kill clash past half month bnp boycott poll call farce ask people stay away statement when call hour begin morning include polling day bnp say one home abroad will recognise election government will appear anew illegal structure more lawmaker already elect unconteste boycott bnp lead alliance other political party say should resign election will rig bnp also accuse government push country civil war when poll date announce protest strike blockade lead

In [114]:
  print(data_list[1030])

Retd Delhi HC Chief Justice and the man behind a landmark verdict decriminalising homosexuality,Justice  A P Shah feels the Supreme Court setting aside that order is unfortunate. At this Idea Exchange moderated by Senior Editor (Legal Affairs) Maneesh Chhibber,he also talks about his new assignment as Law Commission chief,where he is looking into electoral reforms,live-in relationships and age of juvenilityManeesh Chhibber: Can you explain how you wrote your Section 377 judgment?I wouldnt like to comment on the Supreme Court judgment but that doesnt bar me from speaking about the rights of LGBTs,the Constitutional morality we talked about in the high court case,and the governments position. Let me start with this  some speak of this as a western disease. First of all,it is not western. Temple imagery and essential scriptures show there is some evidence of homosexuality being practised in this country… The British brought in Section 377 and there is the presumption that one of the

In [109]:
  print(data_list[3])

The interrogation of two marijuana peddlers about the source of the contraband took the police to a two-and-half-acre plot on Mula-Mutha riverbed in Mundhwa where officers discovered 450 cannabis plants that had been thriving there for the past several months. Based on the probe till now, cops believe that the growth of the cannabis could have happened because of the seed dispersal from initial natural growth.
On February 13, the anti-narcotics cell of Pune police received a tip-off about two persons from Mundhwa allegedly peddling marijuana on a bike. A trap was laid in Mundhwa and Suraj Atole (19) and Nitin Adagale (22) — residents of Mundhwa and working as labourers on construction site in the nearby area — were arrested.
Police initially recovered 1.6 kg marijuana worth Rs 35,000. Cell in-charge Inspector Sunil Tambe said, “Initially, the two were not telling us from where they had been procuring the contraband. But after sustained interrogation, they took us to a spot in the river

In [110]:
  print(data_list[4])

Nine highly decomposed bodies were found stacked on top of each other in a ditch at Pachaspura,six km from the Dimapur DC office complex,Friday afternoon. The victims had been blindfolded,with their hands tied behind and shot in the head from a close range,Karbi Anglong SP Mugdha Jyoti said. One of the bodies has been identified as a Karbi youth from the adjoining Karbi Anglong district in Assam. With the Nagaland police confirming that the bodies bore bullet marks,the Assam Police now suspect that the killings were related to the on-going clash between Karbi and Rengma communities in the Assam district. Meanwhile,a person belonging to the Karbi community was killed when three miscreants opened fire in Kalagaon village under Bokajan police station in Karbi Anglong district around 7 pm Saturday.We have identified one of the bodies to be that of one Harlangbi Ingti (23) of Rongkimi village under Bokajan police station in Karbi Anglong district. He was a student who also had a broiler fa

In [111]:
  print(data_list[24])

With his party having put CM Virbhadra Singh under the scanner for corruption,former chief minister Prem Kumar Dhumal and his MP son Anurag Thakur may find themselves in fresh trouble with the state vigilance set to name them in FIRs.Sources said the investigating agency has got a go-ahead from the government to invoke the strongest provisions in challans proposed to be filed in twin FIRs relating to Himachal Pradesh Cricket Associations (HPCA) irregularities and change of status from a registered society to a company,and demolition of a residential building close to the cricket ground. Anurag is president of HPCA.There is also a third FIR under investigation pertaining to illegal felling of forest trees to built a five-star hotel - The Pavilion - as part of the cricket stadium project. It was decided that challans would be presented in court immediately and all HPCA functionaries and government officials,apart from political leaders,who were found to be involved should also be name